# Notebook that loads a Filtered Signal and Exports it into a format that can be used by the remaining Signal-to-Spike Stages

### Check WD (change if necessary) and file loading

In [22]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

PATH_TO_FILE = '' # 'src/hfo/'  # This is needed if the WD is not the same as the file location

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


## Extract the channel for the 3 frequency bands

In [23]:
from utils.input import RIPPLE_BAND_FILENAME, FR_BAND_FILENAME, BOTH_BAND_FILENAME

ripple_file_suffix = f'{RIPPLE_BAND_FILENAME}_band'
fr_file_suffix = f'{FR_BAND_FILENAME}_band'
hfo_file_suffix = f'{BOTH_BAND_FILENAME}_band'

In [24]:
import numpy as np
import math
from utils.io import preview_np_array

IS_CLINICAL = True
INPUT_PATH = "/clinical" if IS_CLINICAL else "/synthetic"
INPUT_FILE_NAME = "filtered_seeg_csl_ch30-59"    # f"filtered_seeg_ch90-119"

filtered_seeg_path = f"../filter/results{INPUT_PATH}/{INPUT_FILE_NAME}"

ripple_seeg_filename = f"{filtered_seeg_path}_{ripple_file_suffix}.npy"
fr_seeg_filename = f"{filtered_seeg_path}_{fr_file_suffix}.npy"
hfo_seeg_filename = f"{filtered_seeg_path}_{hfo_file_suffix}.npy"

ripple_data = np.load(f"{PATH_TO_FILE}{ripple_seeg_filename}")
fr_data = np.load(f"{PATH_TO_FILE}{fr_seeg_filename}")
hfo_data = np.load(f"{PATH_TO_FILE}{hfo_seeg_filename}")

In [25]:
preview_np_array(ripple_data, "ripple_data", edge_items=2)

preview_np_array(fr_data, "fr_data", edge_items=2)

preview_np_array(hfo_data, "hfo_data", edge_items=2)

ripple_data Shape: (129239, 30).
Preview: [[-8.30193599e-05 -8.93749147e-05 ... -2.82027524e-05  4.44888495e-05]
 [-1.21216938e-03 -1.30402062e-03 ... -4.13498278e-04  6.50108234e-04]
 ...
 [-5.64172868e-01  5.83872137e-01 ...  3.59086578e+00 -2.20422319e+00]
 [-5.98851221e-01  7.19539413e-01 ...  3.87510043e+00 -2.63788951e+00]]
fr_data Shape: (129239, 30).
Preview: [[-1.60660266e-03 -1.72959628e-03 ... -5.45783744e-04  8.60954651e-04]
 [-1.12527184e-02 -1.20958544e-02 ... -3.85575994e-03  6.04031697e-03]
 ...
 [ 3.77408302e-01  5.62804334e-01 ... -3.70504660e-01 -1.01931771e+00]
 [-8.05914205e-01  8.92573648e-01 ... -6.29180359e-01 -7.11263606e-01]]
hfo_data Shape: (129239, 30).
Preview: [[-0.07076982 -0.07618761 ... -0.02404143  0.0379245 ]
 [-0.66374743 -0.71375383 ... -0.22694013  0.35613961]
 ...
 [ 1.11657756 -0.25260712 ...  2.57734347 -0.13053429]
 [ 0.5242394  -0.25160076 ...  0.81962739 -0.06098256]]


## Define Global Parameters of the Experiment

In [26]:
from utils.input import SAMPLING_RATE, X_STEP

num_samples = ripple_data.shape[0]    # 2048 * 120 = 245760
num_channels = ripple_data.shape[1]   # 960

print(f"num_samples: {num_samples}, num_channels: {num_channels}")

num_samples: 129239, num_channels: 30


## Define the Channel we want to extract to feed the SNN

In [27]:
ch_idx = 18

## Fetch the signal for the specified channel

In [28]:
# Extract the seeg_channels in the range [ch_start_idx, ch_end_idx]
ripple_seeg_ch = ripple_data[:, ch_idx]
fr_seeg_ch = fr_data[:, ch_idx]
hfo_seeg_ch = hfo_data[:, ch_idx]

preview_np_array(ripple_seeg_ch, "Ripple sEEG Channel")
preview_np_array(fr_seeg_ch, "FR sEEG Channel")
preview_np_array(hfo_seeg_ch, "HFO sEEG Channel")

Ripple sEEG Channel Shape: (129239,).
Preview: [1.54122064e-04 2.25023101e-03 1.57302684e-02 6.99704806e-02
 2.21330698e-01 ... 3.60463454e-01 1.57081503e+00 2.25298186e+00
 2.13484011e+00 1.23953240e+00]
FR sEEG Channel Shape: (129239,).
Preview: [ 0.00298259  0.02088804  0.03974846 -0.06750419 -0.37735773 ...
  0.50099735 -0.94221199 -1.50319269 -0.31585847  1.35409742]
HFO sEEG Channel Shape: (129239,).
Preview: [ 1.31381294e-01  1.23212452e+00  4.87171836e+00  9.78266318e+00
  7.26022741e+00 ... -4.49401953e+00 -3.09398113e+00 -1.08183782e+00
  1.31395874e-01 -7.54367450e-03]


## Import the Markers (Annotated Events) 
The markers are stored in a numpy array of shape (num_channels, events):
- Each row represents the events of a channel
- Each event is composed of the following 3 fields (Label, Position, Shape)

In [29]:
markers_seeg_file_name = f"{filtered_seeg_path}_markers.npy"
markers = np.load(f"{PATH_TO_FILE}{markers_seeg_file_name}", allow_pickle=True)

print("Markers shape: ", markers.shape)
print("First time steps: ", markers[:10])

Markers shape:  (30,)
First time steps:  [array([('Ripple',   522.9492, 0.), ('Fast Ripple',   523.9258, 0.),
        ('Ripple',  2225.586 , 0.), ('Fast Ripple',  2230.957 , 0.),
        ('Ripple',  2904.2969, 0.), ..., ('Ripple', 60941.895 , 0.),
        ('Fast Ripple', 61286.62  , 0.), ('Ripple', 61297.363 , 0.),
        ('Fast Ripple', 62202.637 , 0.), ('Ripple', 62207.52  , 0.)],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])
 array([('Ripple',   522.9492, 0.), ('Fast Ripple',   523.9258, 0.),
        ('Ripple',  2225.586 , 0.), ('Fast Ripple',  2230.957 , 0.),
        ('Ripple',  2904.2969, 0.), ..., ('Ripple', 60941.895 , 0.),
        ('Fast Ripple', 61286.62  , 0.), ('Ripple', 61297.363 , 0.),
        ('Fast Ripple', 62202.637 , 0.), ('Ripple', 62207.52  , 0.)],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])
 array([('Ripple',   522.9492, 0.), ('Ripple',  2225.586 , 0.),
        ('Fast Ripple',  2230.957 , 0.), ('Ripple',

### Get the markers of the specified channel

In [30]:
relevant_markers = markers[ch_idx]
preview_np_array(relevant_markers, "relevant_markers", edge_items=10)

relevant_markers Shape: (170,).
Preview: [('Ripple',   522.9492, 0.) ('Fast Ripple',   523.9258, 0.)
 ('Ripple',  2225.586 , 0.) ('Fast Ripple',  2230.957 , 0.)
 ('Ripple',  2904.2969, 0.) ('Fast Ripple',  2918.457 , 0.)
 ('Fast Ripple',  3234.8633, 0.) ('Fast Ripple',  3724.121 , 0.)
 ('Ripple',  4879.3945, 0.) ('Ripple',  5058.1055, 0.) ...
 ('Ripple', 59344.24  , 0.) ('Ripple', 60533.69  , 0.)
 ('Ripple', 60738.77  , 0.) ('Fast Ripple', 60741.7   , 0.)
 ('Ripple', 60941.895 , 0.) ('Fast Ripple', 60944.824 , 0.)
 ('Ripple', 61297.363 , 0.) ('Fast Ripple', 61308.105 , 0.)
 ('Fast Ripple', 62202.637 , 0.) ('Ripple', 62207.52  , 0.)]


## Define the Markers of each Band

In [31]:
from utils.input import label_has_fast_ripple, label_has_ripple, label_has_hfo_event

ripple_markers = []
fr_markers = []
hfo_markers = []

for marker in relevant_markers:
    curr_label = marker['label']
    if label_has_ripple(curr_label):
        ripple_markers.append(marker)
    if label_has_fast_ripple(curr_label):
        fr_markers.append(marker)
    if label_has_hfo_event(curr_label):
        hfo_markers.append(marker)

ripple_markers = np.array(ripple_markers)
fr_markers = np.array(fr_markers)
hfo_markers = np.array(hfo_markers)

In [32]:
preview_np_array(ripple_markers, "ripple_markers", edge_items=4)
preview_np_array(fr_markers, "fr_markers", edge_items=4)
preview_np_array(hfo_markers, "hfo_markers", edge_items=4)

ripple_markers Shape: (106,).
Preview: [('Ripple',   522.9492, 0.) ('Ripple',  2225.586 , 0.)
 ('Ripple',  2904.2969, 0.) ('Ripple',  4879.3945, 0.) ...
 ('Ripple', 60738.77  , 0.) ('Ripple', 60941.895 , 0.)
 ('Ripple', 61297.363 , 0.) ('Ripple', 62207.52  , 0.)]
fr_markers Shape: (64,).
Preview: [('Fast Ripple',   523.9258, 0.) ('Fast Ripple',  2230.957 , 0.)
 ('Fast Ripple',  2918.457 , 0.) ('Fast Ripple',  3234.8633, 0.) ...
 ('Fast Ripple', 60741.7   , 0.) ('Fast Ripple', 60944.824 , 0.)
 ('Fast Ripple', 61308.105 , 0.) ('Fast Ripple', 62202.637 , 0.)]
hfo_markers Shape: (170,).
Preview: [('Ripple',   522.9492, 0.) ('Fast Ripple',   523.9258, 0.)
 ('Ripple',  2225.586 , 0.) ('Fast Ripple',  2230.957 , 0.) ...
 ('Ripple', 61297.363 , 0.) ('Fast Ripple', 61308.105 , 0.)
 ('Fast Ripple', 62202.637 , 0.) ('Ripple', 62207.52  , 0.)]


## Export the Processed sEEG and respective markers into a `.npy` file

In [33]:
# To keep the same format as the original data, we will add an extra dimension to the final markers
final_ripple_markers = np.array([ripple_markers])
final_fr_markers = np.array([fr_markers])
final_hfo_markers = np.array([hfo_markers])

preview_np_array(final_ripple_markers, "final_ripple_markers", edge_items=2)
preview_np_array(final_fr_markers, "final_fr_markers", edge_items=2)
preview_np_array(final_hfo_markers, "final_hfo_markers", edge_items=2)

final_ripple_markers Shape: (1, 106).
Preview: [[('Ripple',   522.9492, 0.) ('Ripple',  2225.586 , 0.) ...
  ('Ripple', 61297.363 , 0.) ('Ripple', 62207.52  , 0.)]]
final_fr_markers Shape: (1, 64).
Preview: [[('Fast Ripple',   523.9258, 0.) ('Fast Ripple',  2230.957 , 0.) ...
  ('Fast Ripple', 61308.105 , 0.) ('Fast Ripple', 62202.637 , 0.)]]
final_hfo_markers Shape: (1, 170).
Preview: [[('Ripple',   522.9492, 0.) ('Fast Ripple',   523.9258, 0.) ...
  ('Fast Ripple', 62202.637 , 0.) ('Ripple', 62207.52  , 0.)]]


In [34]:
EXPORT_MARKERS = True

# Define the parent folder to store the results
OUT_PARENT_FOLDER = f"input/{INPUT_FILE_NAME}_ch{ch_idx}"

if EXPORT_MARKERS:
    # Create the folder to store the results if it does not exist
    if not os.path.exists(OUT_PARENT_FOLDER):
        os.makedirs(OUT_PARENT_FOLDER)

    np.save(f"{OUT_PARENT_FOLDER}/markers_{RIPPLE_BAND_FILENAME}_band.npy", final_ripple_markers)   # Save the data to a numpy file (not stored in git due to size)
    np.save(f"{OUT_PARENT_FOLDER}/markers_{FR_BAND_FILENAME}_band.npy", final_fr_markers)
    np.save(f"{OUT_PARENT_FOLDER}/markers_{BOTH_BAND_FILENAME}_band.npy", final_hfo_markers)

In [35]:
# To keep the same format as the original data, we will add an extra inner dimension to the final signal
final_ripple_signal = np.array([ripple_seeg_ch])
final_fr_signal = np.array([fr_seeg_ch])
final_hfo_signal = np.array([hfo_seeg_ch])

preview_np_array(final_ripple_signal, "Final Signal")
preview_np_array(final_fr_signal, "Final Signal")
preview_np_array(final_hfo_signal, "Final Signal")

Final Signal Shape: (1, 129239).
Preview: [[1.54122064e-04 2.25023101e-03 1.57302684e-02 6.99704806e-02
  2.21330698e-01 ... 3.60463454e-01 1.57081503e+00 2.25298186e+00
  2.13484011e+00 1.23953240e+00]]
Final Signal Shape: (1, 129239).
Preview: [[ 0.00298259  0.02088804  0.03974846 -0.06750419 -0.37735773 ...
   0.50099735 -0.94221199 -1.50319269 -0.31585847  1.35409742]]
Final Signal Shape: (1, 129239).
Preview: [[ 1.31381294e-01  1.23212452e+00  4.87171836e+00  9.78266318e+00
   7.26022741e+00 ... -4.49401953e+00 -3.09398113e+00 -1.08183782e+00
   1.31395874e-01 -7.54367450e-03]]


In [36]:
EXPORT_SIGNAL = True
if EXPORT_SIGNAL:
    # Create the folder to store the results if it does not exist
    if not os.path.exists(OUT_PARENT_FOLDER):
        os.makedirs(OUT_PARENT_FOLDER)

    np.save(f"{OUT_PARENT_FOLDER}/{RIPPLE_BAND_FILENAME}_band.npy", final_ripple_signal)   # Save the data to a numpy file (not stored in git due to size)
    np.save(f"{OUT_PARENT_FOLDER}/{FR_BAND_FILENAME}_band.npy", final_fr_signal)
    np.save(f"{OUT_PARENT_FOLDER}/{BOTH_BAND_FILENAME}_band.npy", final_hfo_signal)